## **Chip**

This notebook demonstrates the implementation of the paper - **Chip: CHannel Independence-based Pruning for Compact Neural Networks**

Steps to generate feature maps, calculate channel independence and then compress it are as follows:

Load/create the config file.
Load dataset and create dataloaders.
Load the model along with the weights.
Create Chip object and pass the parameters in the form of a dictionary.
Pass the dataloaders into the compress_model method to obtain the compressed model.
Since this is a demo notebook the number of epochs have been set to 2.

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import sys

sys.path.append("../../../")

import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory
import yaml

/opt/conda/envs/py117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_transform = transforms.Compose([transforms.ToTensor()])

val_transform = transforms.Compose([transforms.ToTensor()])

test_transform = transforms.Compose([transforms.ToTensor()])

transforms1 = {"train": train_transform, "val": val_transform, "test": test_transform}


def train_target_transform(label):
    return label


def val_target_transform(label):
    return label


def test_target_transform(label):
    return label


target_transforms = {"train": None, "val": None, "test": None}


cifar_dataset = DatasetFactory.create_dataset(
    name="CIFAR100",
    root="./",
    split_types=["train", "val", "test"],
    val_fraction=0.2,
    transform=transforms1,
    target_transform=target_transforms,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Getting the configurations.

In [3]:
with open("./chip.yaml", "r") as stream:
    data_loaded = yaml.safe_load(stream)
data_loaded

{'CFG': {'batch_size': 16,
  'arch': 'resnet_50',
  'repeat': 5,
  'ci_dir': './ci_resnet_50',
  'lr_type': 'cos',
  'learning_rate': 0.01,
  'epochs': 2,
  'num_layers': 53,
  'feature_map_dir': './conv_feature_map',
  'sparsity': '[0.]+[0.1]*3+[0.35]*16',
  'label_smooth': 0,
  'device': 'cuda',
  'gpu': 1,
  'momentum': 0.9,
  'weight_decay': 0.005,
  'lr_decay_step': '30,60',
  'result_dir': './result',
  'pretrain_dir': './pretrained_models/resnet_50.pt',
  'conv_index': 1},
 'wandb': True,
 'insize': 32}

## Creating the dataloaders

In [4]:
train_loader = torch.utils.data.DataLoader(
    cifar_dataset["train"],
    batch_size=128,
    sampler=cifar_dataset["train_sampler"],
    num_workers=2,
)
val_loader = torch.utils.data.DataLoader(
    cifar_dataset["val"],
    batch_size=128,
    sampler=cifar_dataset["val_sampler"],
    num_workers=2,
)
test_loader = torch.utils.data.DataLoader(
    cifar_dataset["test"],
    batch_size=128,
    sampler=cifar_dataset["test_sampler"],
    num_workers=2,
)

## Loading the model.

In [5]:
model = ModelsFactory.create_model(
    name="resnet50", version="chip", num_classes=100, **data_loaded["CFG"]
)

## Creating the method's object followed by compression.

In [6]:
from trailmet.algorithms.prune.chip import Chip

In [7]:
a = Chip(
    model,
    {"train": train_loader, "val": val_loader, "test": test_loader},
    **data_loaded["CFG"]
)

wandb: Currently logged in as: animesh-007. Use `wandb login --relogin` to force relogin


In [8]:
a.compress_model()

Calculating feature maps of ResNet50 for block 0


Block 0: 100%|██████████| 3/3 [00:09<00:00,  3.21s/it]


Calculating feature maps of ResNet50 for block 1


Block 1: 100%|██████████| 4/4 [00:09<00:00,  2.44s/it]


Calculating feature maps of ResNet50 for block 2


...

Repeat CI for layer 36: 100%|██████████| 5/5 [02:12<00:00, 26.59s/it]


Caclulating Mean Repeat CI for layer 37


Repeat CI for layer 37: 100%|██████████| 5/5 [00:31<00:00,  6.25s/it]


Caclulating Mean Repeat CI for layer 38


Repeat CI for layer 38:   0%|          | 0/5 [00:00<?, ?it/s]